In [ ]:
# First and foremost, we import pandas.
import pandas as pd

# Next, we import the data from the file inside the folder next to this one.
path = "../Data/Telco_Customer_Churn.csv"
df = pd.read_csv(path)

# Now we confirm if the data upload worked properly.
df.head()

# We must see the DataFrame status:
#df.info()

# We can appreciate that there are no nulls. Suspicious...
# We can see as well that there are some columns in lowercase among the others. 
# This might be optional, but I want them to look orderly.
df.rename(columns = {"customerID": "CustomerID", "gender": "Gender", "tenure" : "Tenure"}, inplace = True)
#df.info()

# The column TotalCharges is type object. Let's format it into float.
# df["TotalCharges"] = pd.to_numeric(df["TotalCharges"])
# An error occurred. Let's see what happened.
df["TotalCharges"].value_counts() # 11 counted as " ".

# Let's find entries without a ".".
df[~df["TotalCharges"].str.contains(r"\.")] # 335 rows with int numbers mixed in them.

# Let's see if by turning the spaces into zeroes, we can turn the column into a numeric.
df.loc[df["TotalCharges"].str.contains(" "), ["TotalCharges"]] = "0"
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"])

# Now that the DataFrame looks better, we can look for duplicates.
df[df.duplicated()] # No duplicates.

# Let's look for anomalous data:
df["Gender"].value_counts() # There are only two genders, but it's better leaving it as it is
df["SeniorCitizen"].value_counts() # It may count as a boolean column: 0-1
df["Partner"].value_counts() # This one might count as boolean as well: Yes/No
df["Dependents"].value_counts() # This one as well: Yes/No
df["PhoneService"].value_counts() # Boolean: Yes/No
df["MultipleLines"].value_counts() # This one has three categories, so it stays as object: No/Yes/No phone service
df["InternetService"].value_counts() # Options: Fiber optic / DSL / No
df["OnlineSecurity"].value_counts() # Thee options: No / Yes / No internet service
df["OnlineBackup"].value_counts() # Three options: No / Yes / No internet service
df["DeviceProtection"].value_counts() # Three options: No / Yes / No internet service
df["TechSupport"].value_counts() # Three options: No / Yes / No internet service
df["StreamingTV"].value_counts() # Three options: No / Yes / No internet service
df["StreamingMovies"].value_counts() # Three options: No / Yes / No internet service
df["Contract"].value_counts() # Three options: Monto to month / Two year / One year
df["PaperlessBilling"].value_counts() # Boolean: Yes / No
df["PaymentMethod"].value_counts() # Four options: Mailed check / Electronic check / Bank transfer (automatic) / Credit card (sutomatic)
df["Churn"].value_counts() # Boolean

# Before the transformation, I want to make sure no spaces escape me.
columns = ["Gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity",
            "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "Contract", "PaperlessBilling",
            "PaymentMethod",  "Churn", "StreamingMovies"]

blank_list = []
for column in columns:
    blanks = df[column].isin([" ", ""]).sum()
    blank_list.append(f"Column {column} has {blanks} blanks.")

print(blank_list)

# There are no blanks, so we can proceed.
# Next step: Turning binary columns into booleans
df["SeniorCitizen"] = df["SeniorCitizen"].astype(bool)
df["Partner"] = df["Partner"] == "Yes"
df["Dependents"] = df["Dependents"] == "Yes"
df["PhoneService"] = df["PhoneService"] == "Yes"
df["PaperlessBilling"] = df["PaperlessBilling"] == "Yes"
df["Churn"] = df["Churn"] == "Yes"

# We check on the DataFrame info now:
#df.info()

# "Tenure", "MonthlyCharges" and "TotalCharges" are numeric, so we have to use describe() to find any anomalies.
df.describe()

# Now, we must fill those previous zeroes by looking at the data and finding a logic worth applying.
df[["Tenure", "MonthlyCharges", "Contract", "TotalCharges"]]

# By the look of it, to fill our zeroes, we must multiply the tenure by the monthly charges to get the total charges...
# Let's see if thats the case by making a temporary column.
df["TempTotalCharges"] =  df["Tenure"] * df["MonthlyCharges"]
df[["Tenure", "MonthlyCharges", "Contract", "TotalCharges","TempTotalCharges"]].tail(20)

# Let's deletethe comparative column
df.drop("TempTotalCharges", axis = 1, inplace = True)

# We can see that the real values get close to what we want to achieve, although it's not perfect...
# It's probable that; whether some clients have a few discounts or additional fees we can't know about, 
# or the company is getting extra money, unbeknownst to their clients, behind their backs.
# Either way, let's get the info from those columns.
df.loc[df["TotalCharges"] == 0]

# Tenure means how many months clients have subscribed to the company's services.
# By looking at this rows, we can see they are new to this contract, so they still don't have any charges
# to pay. That's why they were blank in the first place.


['Column Gender has 0 blanks.', 'Column Partner has 0 blanks.', 'Column Dependents has 0 blanks.', 'Column PhoneService has 0 blanks.', 'Column MultipleLines has 0 blanks.', 'Column InternetService has 0 blanks.', 'Column OnlineSecurity has 0 blanks.', 'Column OnlineBackup has 0 blanks.', 'Column DeviceProtection has 0 blanks.', 'Column TechSupport has 0 blanks.', 'Column StreamingTV has 0 blanks.', 'Column Contract has 0 blanks.', 'Column PaperlessBilling has 0 blanks.', 'Column PaymentMethod has 0 blanks.', 'Column Churn has 0 blanks.', 'Column StreamingMovies has 0 blanks.']


,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,False,True,True,0,False,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,True,Bank transfer (automatic),52.55,0.0,False
753,3115-CZMZD,Male,False,False,True,0,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,20.25,0.0,False
936,5709-LVOEQ,Female,False,True,True,0,True,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,False,Mailed check,80.85,0.0,False
1082,4367-NUYAO,Male,False,True,True,0,True,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,25.75,0.0,False
1340,1371-DWPAZ,Female,False,True,True,0,False,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,False,Credit card (automatic),56.05,0.0,False
3331,7644-OMVMY,Male,False,True,True,0,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,19.85,0.0,False
3826,3213-VVOLG,Male,False,True,True,0,True,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,25.35,0.0,False
4380,2520-SGTTA,Female,False,True,True,0,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,20.00,0.0,False
5218,2923-ARZLG,Male,False,True,True,0,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,True,Mailed check,19.70,0.0,False
6670,4075-WKNIU,Female,False,True,True,0,True,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,False,Mailed check,73.35,0.0,False
